In [38]:
%pip install -Uq sagemaker
%pip install -Uq datasets


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import boto3
import sagemaker
import json
from sagemaker import Model, image_uris, serializers, deserializers
import time
from pathlib import Path

boto3_session=boto3.session.Session()
# boto3_session=boto3.session.Session()

smr = boto3_session.client("sagemaker-runtime") # sagemaker runtime client for invoking the endpoint
sm = boto3_session.client("sagemaker") # sagemaker client for creating the endpoint
s3_rsr = boto3_session.resource("s3")
# role = sagemaker.get_execution_role()  # execution role for the endpoint
role = "arn:aws:iam::152804913371:role/service-role/AmazonSageMaker-ExecutionRole-20200526T152070"

sess = sagemaker.session.Session(boto3_session, sagemaker_client=sm, sagemaker_runtime_client=smr)  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [ ]:
def process_file(input_path, output_path):
    with open(input_path, "r") as f:
        data = json.load(f)
    
    with open(output_path, "w") as f:
        for record in data:
            new_record = {"prompt_id": record["prompt_id"]}
            conversation = []
            for turn in record["messages"]:
                conversation.append({"role": turn["role"], "value": turn["content"]})
        
            new_record["conversations"] = conversation
            f.write(json.dumps(new_record))
            
process_file("hf3k_train.json", "hf3k_train_processed.jsonl")
process_file("hf3k_test.json", "hf3k_test_processed.jsonl")

### Download Model

In [2]:
model_prefix = "Mistral-7B"
s3_model_path = f"s3://{bucket}/{model_prefix}"
!aws s3 sync s3://jumpstart-cache-prod-$region/huggingface-llm/huggingface-llm-mistral-7b/artifacts/inference/v1.0.0/ $s3_model_path

### Download Data and upload to S3

In [11]:
s3_data = sess.upload_data(path="hf3k_train_processed.jsonl", bucket=bucket, key_prefix="data")

In [12]:
from sagemaker.pytorch import PyTorch
from sagemaker.debugger import TensorBoardOutputConfig

# tb_output_config = TensorBoardOutputConfig(s3_output_path=f"s3://{bucket}/llama7b/tensorboard/",
#     container_local_output_path="/opt/ml/output/tensorboard")

# hyperparameters = {
#     "config_file": "qlora-34b.yml",
#     "deepspeed": "axolotl/deepspeed/zero2.json"
# }

hyperparameters = {
    "config": "qlora.yml",
}

# estimator = PyTorch(
#     source_dir = "src",
#     entry_point="axol_launcher_dist.py",
#     sagemaker_session=sess,
#     role=role,
#     instance_count=2, 
#     hyperparameters=hyperparameters,
#     instance_type="ml.g5.2xlarge", 
#     framework_version="2.0.1",
#     py_version="py310",
#     disable_profiler=True,
#     max_run=60*60*24*2,
#     keep_alive_period_in_seconds=3600,
#     tensorboard_output_config=tb_output_config,
#     environment = {"HUGGINGFACE_HUB_CACHE": "/tmp", 
#                     "LIBRARY_PATH": "/opt/conda/lib/",
#                     "TRANSFORMERS_CACHE": "/tmp"}
# )

estimator = PyTorch(
    source_dir = "src",
    entry_point="axolotl/src/axolotl/cli/train.py",
    sagemaker_session=sess,
    role=role,
    instance_count=4, 
    hyperparameters=hyperparameters,
    instance_type="ml.g5.2xlarge", 
    framework_version="2.3.0",
    py_version="py311",
    disable_profiler=True,
    max_run=60*60*24*2,
    keep_alive_period_in_seconds=3600,
    # tensorboard_output_config=tb_output_config,
    environment = {"HUGGINGFACE_HUB_CACHE": "/tmp", 
                    "LIBRARY_PATH": "/opt/conda/lib/",
                    "TRANSFORMERS_CACHE": "/tmp",
                    "NCCL_P2P_LEVEL": "NVL"},
    distribution={"torch_distributed": {"enabled": True}} 
)

In [13]:
estimator.fit({"model": s3_model_path, "train": s3_data})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2024-07-08-18-02-56-581


2024-07-08 18:03:49 Starting - Starting the training job...
2024-07-08 18:03:55 Downloading - Downloading input data................................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-07-08 18:09:25,455 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-07-08 18:09:25,473 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-07-08 18:09:25,484 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-07-08 18:09:25,486 sagemaker_pytorch_container.training INFO     Invoking TorchDistributed...
2024-07-08 18:09:25,486 sagemaker_pytorch_container.training INFO     Invoking user training script.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-07-08 18:09:25,371 sagemaker-training-toolkit INFO     Imported framework sagemake